<a href="https://colab.research.google.com/github/cbonnin88/meditrack/blob/main/dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.0 MB/s eta 0:00:00


In [ ]:
import polars as pl
import numpy as np
from faker import Faker
import random

In [ ]:
fake = Faker()
Faker.seed(42)
np.random.seed(42)
random.seed(42)

# **Generate Users (with dirt)**

In [ ]:
def generate_users(n=3000):
  data_users = []
  for _ in range(n):
    data_users.append({
        'user_id': fake.uuid4(),
        'signup_date': fake.date_between(start_date='-1y', end_date='today'),
        # Dirty Data: Null ages, impossbily high ages
        'age': random.choice([random.randint(18,80), None, 150]),
        # Dirty Data: Missing Emails
        'email': fake.email() if random.random() > 0.1 else None,
        'country': random.choice(['France','Denmark','Ireland']),
        'acquisition_channel': random.choice(['Instagram','Google','Referral','Organic'])
    })
  return pl.DataFrame(data_users)

# **Generate Events**

In [ ]:
def generate_events(users_df, n_events=20000):
  users_ids = users_df['user_id'].to_list()
  event_types = ['app_open','view_doctor','book_apt','cancel_appt','video_call']

  data_events = []
  for _ in range(n_events):
    u_id = random.choice(users_ids)
    data_events.append({
        'event_id': fake.uuid4(),
        'user_id': u_id,
        'event_name': random.choice(event_types),
        'event_timestamp': fake.date_between(start_date='-1y', end_date='today'),
        'platform': random.choice(['iOS','Android','Web'])
    })
  return pl.DataFrame(data_events)

# **Generate Subscriptions**

In [ ]:
def generate_subs(users_df):
  data_subs = []
  for u_id in users_df['user_id']:
    is_churned = random.choice([0,1])
    data_subs.append({
        'user_id': u_id,
        'monthly_fee': random.choice([10.95,20.95]),
        'churned': is_churned
    })
  return pl.DataFrame(data_subs)

# **Execute and Save Raw Data**

In [ ]:
print('Generating Raw Data...')
df_users = generate_users()
df_events = generate_events(df_users)
df_subs =  generate_subs(df_users)

Generating Raw Data...


In [ ]:
df_users.write_csv('meditrack_users_raw.csv')
df_events.write_csv('meditrack_events_raw.csv')
df_subs.write_csv('meditrack_subs_raw.csv')
print('Phase 0 Complete: Raw Files created')

Phase 0 Complete: Raw Files created
